In [ ]:
from readlif.reader import LifFile
import os
import numpy as np
import tifffile
from skimage.transform import resize

def convert_lif_to_tiff_hyperstack(lif_file_path, output_base_dir):
    """
    Convert a LIF file to multipage TIFF files organized by image and tile.
    Each TIFF is saved as a proper hyperstack with separate dimensions for 
    channels and z-slices that ImageJ can directly recognize. The images are 
    also downscaled by a factor of 0.5.
    
    Args:
        lif_file_path (str): Path to the LIF file
        output_base_dir (str): Base directory to save the output
    """
    # Load the LIF file
    lif_file = LifFile(lif_file_path)
    
    # Get the filename without extension for naming the output folders
    file_name = os.path.splitext(os.path.basename(lif_file_path))[0]
    base_output_dir = os.path.join(output_base_dir, file_name)
    
    # Process each image in the LIF file
    for image_idx in range(lif_file.num_images):
        print(image_idx)
        # Get the image
        image = lif_file.get_image(image_idx)
        image.channel_as_second_dim = False
        
        # Create a directory for this image
        image_name = image.name if image.name else f"Image_{image_idx}"
        image_dir = os.path.join(base_output_dir, image_name)
        os.makedirs(image_dir, exist_ok=True)
        
        # Get the number of channels
        num_channels = image.channels
        
        print(f"Processing Image {image_idx}: {image_name}")
        print(f"  Dimensions: z={image.dims.z}, t={image.dims.t}, channels={num_channels}, m={image.dims.m}")
        
        # Process each mosaic tile
        for m in range(image.dims.m):
            try:
                print(f"  Processing tile {m}")
                
                # First, get sample frame to determine dimensions
                sample_frame = None
                for z in range(image.dims.z):
                    for c in range(num_channels):
                        try:
                            sample_frame = image.get_frame(z=z, t=0, c=c, m=m)
                            if sample_frame is not None:
                                break
                        except Exception:
                            pass
                    if sample_frame is not None:
                        break
                
                if sample_frame is None:
                    print(f"  Warning: Could not get any valid frames for tile {m}")
                    continue
                
                # Get dimensions from sample frame
                width, height = sample_frame.size
                new_width, new_height = width // 1, height // 1  # Downscale by 0.5
                
                # Debug: print frame size
                print(f"  Original size: {width} x {height}, Downscaled size: {new_width} x {new_height}")
                
                # Create a list to collect images for each channel and z-slice
                channels_data = []
                
                # For each channel
                for c in range(num_channels):
                    channel_frames = []
                    
                    # For each z-slice
                    for z in range(image.dims.z):
                        try:
                            # Get the frame for this specific channel and z-slice
                            frame = image.get_frame(z=z, t=0, c=c, m=m)
                            
                            # Convert to numpy array (preserving original dtype)
                            frame_array = np.array(frame, dtype=np.uint8)  # Keep original intensity range
                            
                            # Downscale the image with preserve_range=True
                            downscaled_frame = resize(
                                frame_array, 
                                (new_height, new_width), 
                                mode="reflect", 
                                anti_aliasing=True,
                                preserve_range=True  # Keep intensity values unchanged
                            ).astype(np.uint8)  # Convert back to 16-bit (or original dtype)
                            
                            channel_frames.append(downscaled_frame)
                        except Exception as e:
                            print(f"  Warning: Error processing z={z}, c={c}, m={m}: {e}")
                            # Create an empty frame with the downscaled dimensions
                            empty_frame = np.zeros((new_height, new_width), dtype=np.uint8)
                            channel_frames.append(empty_frame)
                    
                    # Stack all z-slices for this channel
                    if channel_frames:
                        channel_stack = np.stack(channel_frames)
                        channels_data.append(channel_stack)
                
                # Check if we have data for all channels
                if len(channels_data) != num_channels:
                    print(f"  Warning: Expected {num_channels} channels but got {len(channels_data)}")
                
                # Stack all channels to create the final hyperstack
                # This should be a 4D array: [channels, z-slices, height, width]
                if channels_data:
                    hyperstack = np.stack(channels_data)
                    
                    # Reshape to [z, c, y, x] for ImageJ
                    # ImageJ expects TZCYX order with T=1
                    hyperstack = np.transpose(hyperstack, (1, 0, 2, 3))
                    
                    # Define output file path
                    tiff_filename = f"{image_name}_tile_{m:04d}.tiff"
                    tiff_path = os.path.join(image_dir, tiff_filename)
                    
                    # Debug: print hyperstack shape
                    print(f"  Hyperstack shape: {hyperstack.shape}")
                    
                    # Save as a proper hyperstack with ImageJ metadata
                    tifffile.imwrite(
                        tiff_path, 
                        hyperstack, 
                        imagej=True,
                        metadata={
                            'axes': 'ZCYX',
                            'channels': num_channels,
                            'slices': image.dims.z
                        }
                    )
                    print(f"  Saved {tiff_path} as hyperstack with {image.dims.z} z-slices × {num_channels} channels")
                    
            except Exception as e:
                print(f"  Error processing tile {m}: {e}")

if __name__ == "__main__":
    # Example usage
    lif_file_path = r"D:\Box Sync\confocal\brain\injections\x250311_brain.lif"
    output_base_dir = r"D:\Box Sync\confocal\brain\injections\TIFF"
    
    convert_lif_to_tiff_hyperstack(lif_file_path, output_base_dir)


0
Processing Image 0: TileScan_001
  Dimensions: z=39, t=1, channels=3, m=1
  Processing tile 0
  Original size: 512 x 512, Downscaled size: 256 x 256
  Hyperstack shape: (39, 3, 256, 256)
  Saved D:\confocal\shark_retina\wholemount\TIFF\s240913_shark_DAPI_calbin_GABA\TileScan_001\TileScan_001_tile_0000.tiff as hyperstack with 39 z-slices × 3 channels
1
Processing Image 1: TileScan_002
  Dimensions: z=39, t=1, channels=3, m=1482
  Processing tile 0
  Original size: 512 x 512, Downscaled size: 256 x 256
  Hyperstack shape: (39, 3, 256, 256)
  Saved D:\confocal\shark_retina\wholemount\TIFF\s240913_shark_DAPI_calbin_GABA\TileScan_002\TileScan_002_tile_0000.tiff as hyperstack with 39 z-slices × 3 channels
  Processing tile 1
  Original size: 512 x 512, Downscaled size: 256 x 256
  Hyperstack shape: (39, 3, 256, 256)
  Saved D:\confocal\shark_retina\wholemount\TIFF\s240913_shark_DAPI_calbin_GABA\TileScan_002\TileScan_002_tile_0001.tiff as hyperstack with 39 z-slices × 3 channels
  Processin